In [1]:
import carla
import random
import queue
import numpy as np
import cv2
import pandas as pd

In [2]:
def gen_car(world):
    bp_lib = world.get_blueprint_library()
    spawn_points = world.get_map().get_spawn_points()
    vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')

    test = True
    while test:
        vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
        if vehicle:
            test = False
            vehicle.set_autopilot(True)

    camera_r = bp_lib.find('sensor.camera.rgb')
    # camera_r.set_attribute('image_size_x', '640')
    # camera_r.set_attribute('image_size_y', '480')
    camera_init_trans = carla.Transform(carla.Location(z=2))
    camera_rgb = world.spawn_actor(camera_r, camera_init_trans, attach_to=vehicle)

    camera_d = bp_lib.find('sensor.camera.depth')
    # camera_d.set_attribute('image_size_x', '640')
    # camera_d.set_attribute('image_size_y', '480')
    camera_init_trans = carla.Transform(carla.Location(z=2))
    camera_depth = world.spawn_actor(camera_d, camera_init_trans, attach_to=vehicle)

    camera_s = bp_lib.find('sensor.camera.semantic_segmentation')
    # camera_s.set_attribute('image_size_x', '640')
    # camera_s.set_attribute('image_size_y', '480')
    camera_init_trans = carla.Transform(carla.Location(z=2))
    camera_semantic = world.spawn_actor(camera_s, camera_init_trans, attach_to=vehicle)


    rgb_queue = queue.Queue()
    camera_rgb.listen(rgb_queue.put)
    depth_queue = queue.Queue()
    camera_depth.listen(depth_queue.put)
    semantic_queue = queue.Queue()
    camera_semantic.listen(semantic_queue.put)

    return vehicle, rgb_queue, depth_queue, semantic_queue, camera_rgb, camera_depth, camera_semantic

In [3]:
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library()
# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 5
world.apply_settings(settings)
vehicle, rgb_queue, depth_queue, semantic_queue, camera_rgb, camera_depth, camera_semantic = gen_car(world)

In [4]:
world.tick()
image = semantic_queue.get()
img_semantic = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [5]:
arr_depth = img_semantic[:, :, 2]
arr_depth[arr_depth != 10] = 0
arr_depth[arr_depth == 10] = 1

In [6]:
np.save("data/remove_car.npy", arr_depth)

In [7]:
arr_depth == 1

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])